# Datasets

In this notebook, you will be guided to :
- Create and upload **datasets** (image, rasters, point cloud, mesh, vector and file)
- Create an **annotation**

#### You may execute this notebook one cell after the other with `Shift + Enter`

For more options, explore `Cell` in the top menu bar, or read a [tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial/)

In [ ]:
import os
import alteia

In [ ]:
import getpass
platform_url = 'https://app.alteia.com'
login = input('Enter your email ')
password = getpass.getpass('Enter your password ')

In [ ]:
sdk = alteia.SDK(url=platform_url, user=login, password=password)

my_project_id = '65b91245e98f3bab050e2010'
my_mission_id = '65b91530470dd8efd8cd76c7'

In [ ]:
# Set working directory
sample_path = './Banana'
%cd {sample_path}

## Create the raster datasets

### Orthomosaic

In [ ]:
ortho_dataset = sdk.datasets.create_raster_dataset(
  name='Orthomosaic',
  project=my_project_id,
  mission=my_mission_id,
  dataset_format='geotiff',
  categories=['orthomosaic'])

sdk.datasets.upload_file(
  dataset=ortho_dataset.id,
  component='raster',
  file_path='Orthomosaic.tif')

### DSM (Digital Surface Model)

In [ ]:
dsm_dataset = sdk.datasets.create_raster_dataset(
  name='DSM',
  project=my_project_id,
  mission=my_mission_id,
  dataset_format='geotiff',
  categories=['dsm'])

sdk.datasets.upload_file(
  dataset=dsm_dataset.id,
  component='raster',
  file_path='DSM.tif')

### Create the point cloud dataset

In [ ]:
pcl_dataset = sdk.datasets.create_pcl_dataset(
  name='PointCloud',
  project=my_project_id,
  mission=my_mission_id,
  dataset_format='las')

sdk.datasets.upload_file(
  dataset=pcl_dataset.id,
  component='pcl',
  file_path='Point Cloud.las')

### Create the mesh dataset

In [ ]:
mesh_dataset = sdk.datasets.create_mesh_dataset(
  name='Mesh',
  project=my_project_id,
  mission=my_mission_id,
  dataset_format='obj',
  texture_count=1)

sdk.datasets.upload_file(
  dataset=mesh_dataset.id,
  component='mesh',
  file_path='Model.obj')

sdk.datasets.upload_file(
  dataset=mesh_dataset.id,
  component='material',
  file_path='Model.mtl')

sdk.datasets.upload_file(
  dataset=mesh_dataset.id,
  component='texture',
  file_path='Model_0.jpg')

### Create the vector dataset

In [ ]:
vector_dataset = sdk.datasets.create_vector_dataset(
    name='ContoursLines',
    project=my_project_id,
    mission=my_mission_id,
    dataset_format='geojson')

sdk.datasets.upload_file(
  dataset=vector_dataset.id,
  component='vector',
  file_path='Contours.geojson')

In [ ]:
vector_dataset_ing = sdk.datasets.describe(vector_dataset.id)
vector_dataset_ing.__dict__

In [ ]:
# Add category
sdk.datasets.add_categories(vector_dataset_ing.id, categories=['contour lines'])

#### Now, we can see the specificity of type vector

In [ ]:
# Search vector dataset on your project / mission
lst_vct = sdk.datasets.search(
    filter={
        'project': {'$eq': my_project_id},
        'mission': {'$eq': my_mission_id},
        'type': {'$eq': 'vector'},
        'ingestion.status': {'$eq': 'completed'}
    }
)
for vct in lst_vct:
    print(f'Dataset vector with id {vct.id} and name {vct.name} has a source {vct.source["name"]}')

In [ ]:
# There is another solution to find the map-service dataset
vector_dataset_mp = sdk.datasets.search(
    filter={
        "components.origin.id": {
            "$eq": vector_dataset_ing.id}
    }
)
vector_dataset_mp = vector_dataset_mp[0]
vector_dataset_mp.__dict__

In [ ]:
# Add category
sdk.datasets.add_categories(vector_dataset_mp.id, categories=['contour lines'])
# Get the collection to get the features
vector_dataset_mp.components

In [ ]:
collection_id = vector_dataset_mp.components[0]['collection']['id']
print(collection_id)
collection = sdk.collections.describe(collection_id)
collection.__dict__

In [ ]:
collection.feature_count

In [ ]:
lst_features_id = collection.features

In [ ]:
for f in lst_features_id:
    feat = sdk.features.describe(f)
    properties = feat.properties
    geom = feat.geometry
    break
print(properties)
print(geom)

### Create an annotation

In [56]:
my_annotation = sdk.annotations.create(
   project=my_project_id,
   mission=my_mission_id,
   geometry={
       "type": "Point",
       "coordinates": [12.397626065125216, -6.001589143208957]
   },
   name='Annotation_4.0',
   type='2d',
   icon=sdk.annotations.Icons.CONVEYOR
)

### Create an annotation with an image attached

In [57]:
default_horizontal_srs_wkt = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY\
["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],\
UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

In [59]:
image_ann_dataset = sdk.datasets.create_image_dataset(
  name='00001504',
  project=my_project_id,
  mission=my_mission_id,
  geometry={"type": "Point",
            "coordinates": [12.39829438, -6.00186998]},
  horizontal_srs_wkt=default_horizontal_srs_wkt,
  width=2448,
  height=2048
)

In [60]:
sdk.datasets.upload_file(
  dataset=image_ann_dataset.id,
  component='image',
  file_path='00001504.jpg')

In [62]:
my_annotation = sdk.annotations.create(
   project=my_project_id,
   mission=my_mission_id,
   geometry={
       "type": "Point",
       "coordinates": [12.39829438, -6.00186998]
   },
   name='Annotation_4.0_img_v2',
   attachments=[image_ann_dataset.id],
   type='2d',
   icon=sdk.annotations.Icons.PARKING
)